# Python Tutorial: Quickly Make Beautiful Interactive Maps - Zoom Out

This is the third part in our four part series on how to use python to quickly create interactive maps.   

In part 1 we demonstrated how to create a map object, locate it, decide the zoom range and tiles.     
In part 2 we showed how to style and put groups of markers and circles on a map. 

Here we focus on layer features used most commonly in a zoomed-out map:

 
* Heatmaps: static or changing with time.
* Choropleths  

We conclude with a discussion on which feature is best for different use cases (i.e, choosing between markers, circles, heatmaps, choropleths).


**Disclaimer**  
He we discuss only a handful of features which we use frequently. The reader is encouraged to use the module help pages to explore the full potential and updates in the API. 

Maps used are provided by © OpenStreetMap contributors. 

In [1]:
import folium  # to install: pip install folium

folium.__version__

u'0.5.0'

## Data  

To set the stage we will introduce some data that we want to put on a map. Here we us `pandas`, but this is not required (`lists` and `tuples` work fine, too).  
(We used [Google Maps API](`https://maps.googleapis.com`) to pull the actual locations, all located in New York City)   

In [2]:
places = ['4 Washington Place',
 '10 Washington Place',
 '16 Washington Place',
 '22 Washington Place',
 '28 Washington Place',
 '34 Washington Place',
 '40 Washington Place',
 '46 Washington Place',
 '64 Washington Place',
 '70 Washington Place',
 '76 Washington Place',
 '82 Washington Place',
 '88 Washington Place',
 '94 Washington Place']


latitude_longitudes = [(40.729020400000003, -73.99428429999999),
 (40.729370000000003, -73.995001299999998),
 (40.729528100000003, -73.995153799999997),
 (40.729940999999997, -73.995627599999992),
 (40.729841800000003, -73.995996199999993),
 (40.7300495, -73.995843800000003),
 (40.730065400000001, -73.995875500000011),
 (40.729164500000003, -73.994440999999995),
 (40.731659499999999, -73.999168999999995),
 (40.731653999999999, -73.999519599999999),
 (40.731793499999988, -73.999886799999999),
 (40.731885699999999, -74.000196599999995),
 (40.7319855, -74.000376799999998),
 (40.732303399999999, -74.000484200000002)]

In [3]:
import pandas as pd

df_ = pd.DataFrame({'latitude_longitude': latitude_longitudes}, index=places)

df_.index.name = 'place'
df_.head(4)

,latitude_longitude
place,
4 Washington Place,"(40.7290204, -73.9942843)"
10 Washington Place,"(40.72937, -73.9950013)"
16 Washington Place,"(40.7295281, -73.9951538)"
22 Washington Place,"(40.729941, -73.9956276)"


## HeatMap

A `folium.plugins.HeatMap` object is all you need to create a heatmap and is added as a layer to your `Map` object.  
 
Useful keywords:  
* `radius` - radius of each point (in pixels)
* `blur` - how much to blur the points for a visual effect
* `gradient` - Color gradient for blobs (default `None`) a dictionary of values. E,g `{0.4: 'blue', 0.65: 'lime', 1.: 'red'}`

In [4]:
from folium import plugins

In [5]:
# ====== Create a map object, location and zoom ======
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
zoom_start = 16

map_ = folium.Map(location=location, zoom_start=zoom_start, control_scale=True) 

# ====== Create the HeatMap layer ======
radius_pixels = 30
blur = 20
gradient = None 
heatmap = plugins.HeatMap(latitude_longitudes, name = "Heatmap", radius=radius_pixels, blur=blur, gradient=gradient)

# ====== Add Heatmap Layer to the map ======
heatmap.add_to(map_)

# ====== Display ======
# map_.save('my_map.html') # (alternatively create HTML)

map_ # display in notebook

Another cool option is a heatmap as a function of time `folium.plugins.HeatMapWithTime`.  
This is useful to show how a data evolves as a function of time, e.g, to follow trend or an epidemic. 

For this purpose you will want to create a list of lists of locations, where the locations have to be a list of longitude-latitude (not a tuple like before).   

In [14]:
# Example data
l_points = map(lambda x: list(x),  latitude_longitudes) # must be a list of list of lists

l_l_points = []
for i in range(len(l_points)/2):
    l_l_points.append( l_points[:(i + 1)] + l_points[-(i+2):-1])

# steps names (needs to be same length as l_l_points)
index_steps  = ["Monday", 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    
print "Note that the data input is a list of lists of lists\n"
print "In our example\n`l_l_points` contains {} step lists of geo coordinates, \nwhere the last list contains {} geo locations, \nwhere each geo location contains {} coordinates".format(len(l_l_points), len(l_l_points[-1]), len(l_l_points[-1][0]))
print "The {} steps are labeled as: {}".format(len(l_l_points), index_steps)
print '-' * 20
print "E.g, the step labelled {} displays geo locations that are in a list of lists:".format(index_steps[-1])
l_l_points[-1]

Note that the data input is a list of lists of lists

In our example
`l_l_points` contains 7 step lists of geo coordinates, 
where the last list contains 14 geo locations, 
where each geo location contains 2 coordinates
The 7 steps are labeled as: ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
--------------------
E.g, the step labelled Sunday displays geo locations that are in a list of lists:


[[40.7290204, -73.99428429999999],
 [40.72937, -73.9950013],
 [40.7295281, -73.9951538],
 [40.729941, -73.99562759999999],
 [40.7298418, -73.9959962],
 [40.7300495, -73.9958438],
 [40.7300654, -73.99587550000001],
 [40.7300654, -73.99587550000001],
 [40.7291645, -73.994441],
 [40.7316595, -73.999169],
 [40.731654, -73.9995196],
 [40.73179349999999, -73.9998868],
 [40.7318857, -74.0001966],
 [40.7319855, -74.0003768]]

In [15]:
radius_pixels = 100
auto_play = True # True means that the animation will start automatcally
heatmap_wTime = plugins.HeatMapWithTime(l_l_points, index_steps, radius=radius_pixels, auto_play=auto_play, display_index=True)

# ====== Create a map object, location and zoom ======
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
zoom_start = 16

map_ = folium.Map(location=location, zoom_start=zoom_start) 


# ====== Add Marker Group Layer to the map ======
heatmap_wTime.add_to(map_)

# ====== Display ======
map_.save('my_map.html') # (alternatively create HTML)

map_ # display in notebook

Notes: 
* Set `auto_play=True` to have the animation start automatically.    
* The user has a loop button (no toggle option currently available in the 0.5.0 API)   


## Choropleths

Choropleths display metrics of a theme by region, e.g ZIP Code income per capita, County unemployement rate.  

Here we show how to create a choropleth for a given Shapefile or GeoJson.  

This section is divided into two parts 

* Reading and manipulating the shape file data  
* From GeoJson to Choropleth in 3.5 steps  

### Shape Data

For the purpose of this example we use the US 2010 Census New York state ZIP Code shapefiles `tl_2010_36_zcta510` found [here](https://www.census.gov/cgi-bin/geo/shapefiles2010/main).  
(It's actually Census ZCTA which is similar to US Post ZIP Code, see [Wikipedia](https://en.wikipedia.org/wiki/ZIP_Code_Tabulation_Area) for details.)

Here we:  
* Open and read the shape file with  `shapefile` from [pyshp](https://pypi.python.org/pypi/pyshp) 
* Create a geoJson file with  `json.dumps`  
* Read the geoJson file with [geopandas](http://geopandas.org/).  
This yields a `GeoDataFrame` which is simlar in usage to the `pandas.DataFrame`  
* Sample a handful of ZIP Codes to explore and examine them on a map


### ShapeFile to GeoJson

In [17]:
import shapefile # pip install pyshp (for opening the shape file)
from json import dumps # (for creating a geoJSON file)
import geopandas # pip install geopandas (for reading a geoJSON file)

In [26]:
# Defining file paths 

shape_path = "../data/tl_2010_36_zcta510/tl_2010_36_zcta510.shp"
geojson_path = "../../beautiful_interactive_maps/notebooks/aux/tl_2010_36_zcta510.geojson"

In [11]:
# Opening the Shape File

reader = shapefile.Reader(shape_path)
fields = reader.fields[1:]
field_names = [field[0] for field in fields]

buffer_ = []
for sr in reader.shapeRecords():
       atr = dict(zip(field_names, sr.record))
       geom = sr.shape.__geo_interface__
       buffer_.append(dict(type="Feature", geometry=geom, properties=atr)) 

In [12]:
# Creating a geoJSON file 

geojson = open(geojson_path, "w")
geojson.write(dumps({"type": "FeatureCollection", "features": buffer_}, indent=2) + "\n")
geojson.close()

In [27]:
# Reading in the geoJSON to a GeoDataFrame

gdf_geo = geopandas.read_file(geojson_path)
gdf_geo.set_index('ZCTA5CE10', inplace=True) # ZIP Codes are unique, hence setting as index
print gdf_geo.shape
gdf_geo.head(6)

(1794, 11)


,AWATER10,PARTFLG10,CLASSFP10,ALAND10,STATEFP10,FUNCSTAT10,INTPTLAT10,MTFCC10,GEOID10,INTPTLON10,geometry
ZCTA5CE10,,,,,,,,,,,
12205,243508,N,B5,40906445,36,S,+42.7187855,G6350,3612205,-073.8292399,"POLYGON ((-73.87051699999999 42.751227, -73.86..."
12009,2168637,N,B5,135241924,36,S,+42.6975663,G6350,3612009,-074.0355422,"POLYGON ((-74.10891099999999 42.653004, -74.10..."
14804,232123,N,B5,144718714,36,S,+42.3172588,G6350,3614804,-077.8479358,"POLYGON ((-77.92747 42.34775399999999, -77.926..."
14836,131305,N,B5,77612958,36,S,+42.5429182,G6350,3614836,-077.8781933,"(POLYGON ((-77.95599299999999 42.474325, -77.9..."
14536,425175,N,B5,47193482,36,S,+42.5439751,G6350,3614536,-078.0836709,"POLYGON ((-78.050297 42.538502, -78.0502429999..."
10464,236605,N,B5,9070627,36,S,+40.8677868,G6350,3610464,-073.7999204,"(POLYGON ((-73.78619399999999 40.873886, -73.7..."


### Examining the Shapes
Before creating the Choropleth, let's make sure the shape files display nicely on the map.

In [29]:
# ====== GeoDataFrame sample ======
l_zips = ['10002', '10003', '10009','10011', '10012', '10013','10014', '11211', '11222']
gdf_sample = gdf_geo.loc[l_zips]

# ====== initiating map ======
zoom_start = 14
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
map_ = folium.Map(location=location, zoom_start=zoom_start, control_scale=True)

# ====== GeoJson object ======
geojson_ = folium.GeoJson(gdf_sample) # example GeoJson used

geojson_.add_to(map_) # adding to Map object as layer

# ====== Display ======
# map_.save('my_map.html') # (alternatively create HTML)
map_

### Creating a Choropleth in 3.5 steps

* Create a mapping from shape ID to value to color     
* Set `style_function` - maps between IDs and colors.  
Styles features such as opacity, line color  
* Create a `folium.GeoJson` object and layer add to a map (with optional color bar)   
* (Optional) Set a `highlight_function` - this enable highlighting a region when hovered over with the mouse.  

In [33]:
# Step 1: Create a mapping between the ID of a shape to a color
dict_id2color =    { '10002': 'green',
                     '10003': 'green',
                     '10009': 'yellow',
                     '10011': 'red',
                     '10012': 'yellow',
                     '10013': 'green',
                     '10014': 'red',
                     '11211': 'red',
                     '11222': 'yellow'}


# Step 2: Create a style_function  
def my_style_function(feature):
    return {
        'fillColor': dict_id2color[feature['id']],
        'color': 'black', # line color
        'weight': 2,      # line width
        'opacity': 0.7,   # line opacity 
        'fillOpacity': 0.7  # fill color opacity
    }    

The main aspect in `my_style_function` is obtaining the color for each `feature['id']`, where `feature` is a shape.  

Now that we set up the style_function let's see what it looks like in a map.  

In [34]:
# ====== initiating map ======
zoom_start = 14
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
map_ = folium.Map(location=location, zoom_start=zoom_start)


# ==== Creating GeoJson object with style_function ======
geojson_ = folium.GeoJson(gdf_sample,
                          style_function= my_style_function
                         ) 
# ==== Add GeoJson to map as layer ======
map_.add_child(geojson_) # Note that this is the same as geojson_.add_to(map_)

#map_ = map_.add_child(colors_linear) # and the color bar

# ====== Display ======
# map_.save('my_map.html') # (alternatively create HTML)

map_

This is a good first step, but you might also want to add to more options:  
* Mouse-over-shape highlighting      
* Legend  
(these are the optional "half steps" implied in the title)  

For the Mouse-over-shape highlight, all we need to to create a highlight_function. 

In [35]:
# ======= highlight_function ============ 
def my_highlight_function(feature):
    return {
        'fillColor': dict_id2color[feature['id']],
        'color': 'white',
        'weight': 3,
        'dashArray': '0, 0',
        'opacity': 1.,
        'fillOpacity': 1. 
    }


# ====== initiating map ======
zoom_start = 14
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
map_ = folium.Map(location=location, zoom_start=zoom_start)


# ==== Creating GeoJson object with style_function ======
geojson_ = folium.GeoJson(gdf_sample, 
                          style_function= my_style_function, 
                          highlight_function=my_highlight_function
                         ) 
# ==== Add GeoJson to map as layer ======
map_.add_child(geojson_) # Note that this is the same as geojson_.add_to(map_)

#map_ = map_.add_child(colors_linear) # and the color bar

# ====== Display ======
# map_.save('my_map.html') # (alternatively create HTML)

map_

Hover the mouse above an area and see it highlight by your customization.   


For the legend there is no obvious API feature, so here we'll learn how to create a `folium.Element` in JavaScript to add.   

In [200]:
legend_html = """ 
<div style="position: fixed;background-color: rgba(255, 255, 255, 0.5);
    border-radius: 5px;
     bottom: 20px; left: 5px; width: 100px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     ">
     <p style="text-align:center;"> <b> Legend </b> </p>
     <p style="margin-left:5px">Oscar <i class="fa fa-square fa-1x" style="margin-right:5px;color:green; float:right;"></i></p>
     <p style="margin-left:5px">Big Bird <i class="fa fa-square fa-1x" style="margin-right:5px;color:yellow; float:right;"></i></p>
     <p style="margin-left:5px">Elmo <i class="fa fa-square fa-1x" style="margin-right:5px; color:red; float:right;"></i></p>
  </div>
     """



legend_html = """ 
     <div style="position: fixed;background-color: rgba(255, 255, 255, 0.5);
     border-radius: 5px;  
     bottom: 10px; left: 5px; width: 100px; height: 90px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     ">&nbsp; &nbsp; &nbsp; <b>Legend</b> <br> 
     &nbsp;  <i class="fa fa-square fa-1x" style="color:green"></i> Oscar  <br>
     &nbsp;  <i class="fa fa-square fa-1x" style="color:yellow"></i> Big Bird <br>
     &nbsp; <i class="fa fa-square fa-1x" style="color:red"></i>  Elmo  
      </div>
     """


legend_layer = folium.Element(legend_html)


**JaveScript tips**  
`<b> ... </b>` - is boldface  


In [201]:

# ======= highlight_function ============ 
def my_highlight_function(feature):
    return {
        'fillColor': dict_id2color[feature['id']],
        'color': 'white',
        'weight': 3,
        'dashArray': '0, 0',
        'opacity': 1.,
        'fillOpacity': 1. 
    }

# ====== initiating map ======
zoom_start = 14
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
map_ = folium.Map(location=location, zoom_start=zoom_start)


# ==== Creating GeoJson object with style_function ======
geojson_ = folium.GeoJson(gdf_sample, 
                          style_function= my_style_function, 
                          highlight_function=my_highlight_function
                         ) 
# ==== Add GeoJson to map as layer ======
map_.add_child(geojson_) # Note that this is the same as geojson_.add_to(map_)

# Adding Legend
map_.get_root().html.add_child(legend_layer)

# ====== Display ======
# map_.save('my_map.html') # (alternatively create HTML)

map_

In [141]:
# Create a mapping from metric value to color
# Here we use a linear mapping, but there are other options

#import branca.colormap as cm # branca should be installed with folium


l_colors = [  # yellow to red
'#ffffb2',
'#fed976',
'#feb24c',
'#fd8d3c',
'#fc4e2a',
'#e31a1c',
'#b10026']

# choose minimum and maximum values they rperprse
vmin = 0
vmax = 100

colors_linear = cm.LinearColormap(l_colors,  vmin=vmin, vmax=vmax)


colors_linear.caption = "my metric"

colors_linear

In [142]:
# Here we show that metrics out of range get assigned the extreme most colors  
print colors_linear(55) # linear interpolation - not in the original list
print colors_linear(150) # above range get vmax
print colors_linear(-150) # below range get vmax

#fd7a36
#b10026
#ffffb2


More on coding is described in [this tutorial](https://github.com/python-visualization/folium/blob/master/examples/Colormaps.ipynb)  

In [143]:
# Create a mapping from entry ID to color (via ID to metric)
import numpy as np

# ==== You Metric Code Here ====
gdf_sample.loc[:, 'my_metric'] = np.random.uniform(vmin, vmax, len(gdf_sample))
gdf_sample['my_metric_color'] = gdf_sample['my_metric'].map(colors_linear)
# ==============================

# and creating a dictionary mapping the ID to the metric
dict_id2color = gdf_sample['my_metric_color'].to_dict()

dict_id2color

{'10002': '#fd7635',
 '10003': '#fec35e',
 '10009': '#feb953',
 '10011': '#fec05b',
 '10012': '#f84527',
 '10013': '#fec35e',
 '10014': '#c70b21',
 '11211': '#fec15c',
 '11222': '#fed875'}

In [92]:
# (Optional) Set a highlight_function - an interactive highlighting feature

# ====== initiating map ======
zoom_start = 14
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
map_ = folium.Map(location=location, zoom_start=zoom_start)

# ====== Creating GeoJson object with style_function ======

def my_highlight_function(feature):
    return {
        'fillColor': dict_id2color[feature['id']],
        'color': 'white',
        'weight': 3,
        'dashArray': '0, 0',
        'opacity': 1.,
        'fillOpacity': 1. 
    }


geojson_ = folium.GeoJson(gdf_sample,name='bah', 
                          style_function=my_style_function, # (my_style_function is defined in the previous cell)
                          highlight_function=my_highlight_function
                         ) 


# ==== displaying the colored GEOJSON ======
map_.add_child(geojson_)
# map_.add_child(colors_linear) # and the color bar

# ====== Display ======
# map_.save('my_map.html') # (alternatively create HTML)
legend_html = """ 
     <div style="position: fixed;background-color: rgba(255, 255, 255, 0.5);
    border-radius: 5px;
     bottom: 20px; left: 5px; width: 100px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     ">&nbsp; &nbsp; &nbsp; <b>Legend</b> <br> 
     &nbsp;  <i class="fa fa-square fa-2x" style="color:green"></i> Oscar &nbsp; &nbsp; <br>
     &nbsp;  <i class="fa fa-square fa-2x" style="color:yellow"></i> Big Bird &nbsp; <br>
     &nbsp; <i class="fa fa-square fa-2x" style="color:red"></i>  Elmo &nbsp; &nbsp; &nbsp; 
      </div>
     """
map_.get_root().html.add_child(folium.Element(legend_html))
map_.add_child(folium.LayerControl(position='topleft', collapsed=True))

map_

In [125]:
# (Optional) Set a highlight_function - an interactive highlighting feature

# ====== initiating map ======
zoom_start = 14
center_latitude, center_longitude = 40.729183, -73.994263
location = (center_latitude, center_longitude)
map_ = folium.Map(location=location, zoom_start=zoom_start)

# ====== Creating GeoJson object with style_function ======

def my_highlight_function(feature):
    return {
        'fillColor': dict_id2color[feature['id']],
        'color': 'white',
        'weight': 3,
        'dashArray': '0, 0',
        'opacity': 1.,
        'fillOpacity': 1. 
    }


geojson_ = folium.GeoJson(gdf_sample,name='bah', 
                          style_function=my_style_function, # (my_style_function is defined in the previous cell)
                          highlight_function=my_highlight_function
                         ) 


# ==== displaying the colored GEOJSON ======
map_.add_child(geojson_)
# map_.add_child(colors_linear) # and the color bar

# ====== Display ======
# map_.save('my_map.html') # (alternatively create HTML)
legend_html = """ 
<div style="position: fixed;background-color: rgba(255, 255, 255, 0.5);
    border-radius: 5px;
     bottom: 20px; left: 5px; width: 100px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     ">
     <p style="text-align:center;"> <b> Legend </b> </p>
     <p style="margin-left:5px">Oscar <i class="fa fa-square fa-2x" style="margin-right:5px;color:green; float:right;"></i></p>
     <p style="margin-left:5px">Big Bird <i class="fa fa-square fa-2x" style="margin-right:5px;color:yellow; float:right;"></i></p>
     <p style="margin-left:5px">Elmo <i class="fa fa-square fa-2x" style="margin-right:5px; color:red; float:right;"></i></p>
  </div>
     """
map_.get_root().html.add_child(folium.Element(legend_html))
map_.add_child(folium.LayerControl(position='topleft', collapsed=True))

map_




`z-index=9999`  means that we are setting as the layer number 9999 (in our case, the top layer)  

Note that you can also customise the dictionary directly:  
```
dict_id2color = {'10002': 'purple',
                 '10003': 'orange',
                 '10009': 'red',
                 '10011': 'green',
                 '10012': 'yellow',
                 '10013': 'blue',
                 '10014': '#fd7936',
                 '11211': '#93a0b6',
                 '11222': '#339e5f'}
```

## Discussion: Markers, Heatmap or Choropleth?

When should each be used.  
The really depends on the story that you want to tell with the data.  

As for interpretation the choropleth might be more straight forward, because the heatmap color coding requires some work to calculate and convey. 

Both also have tradeoffs of levels of details vs. file size.  

In the case of the choropleth, the more granular you go, you provide more information, but the shape files might be larger in  expensive.  
In the case of the heatmap, the more geo points used the more  


------    
Mention also

Each marker object is about 1kByte in size, meaning that you'd probably want to limit yourself to 1,000 or 10,000 markers.   
